In [12]:
import xml.etree.ElementTree as ET
import glob
import os
import json
from tqdm import tqdm

In [13]:
path='Path to folder'

In [14]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [15]:
classes = []
input_dir = path+"annotations\\xmls\\"
output_dir = path+"Yolo\\"
image_dir = path+"images\\"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [16]:
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in tqdm(files):
    try:
        basename = os.path.basename(fil)
        filename = os.path.splitext(basename)[0]
        if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
            print(f"{filename} image does not exist!")
            continue
        result = []
        tree = ET.parse(fil)
        root = tree.getroot()
        width = int(root.find("size").find("width").text)
        height = int(root.find("size").find("height").text)
        for obj in root.findall('object'):
            label = obj.find("name").text
            if label not in classes:
                classes.append(label)
            index = classes.index(label)
            pil_bbox = [int(x.text) for x in obj.find("bndbox")]
            yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
            bbox_string = " ".join([str(x) for x in yolo_bbox])
            result.append(f"{index} {bbox_string}")
        if result:
            with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
                f.write("\n".join(result))
    except:
        pass

  0%|          | 0/2829 [00:00<?, ?it/s]

100%|██████████| 2829/2829 [03:25<00:00, 13.76it/s]
